## reference
* https://medium.com/@lhartikk/a-blockchain-in-200-lines-of-code-963cc1cc0e54#.f2isajv22
* blockchain for authentication http://www.talkcrypto.org/blog/portfolio/digital-signatures-and-the-blockchain/

## todo

* broadcast updates to blockchain?

In [4]:
from PyBlockchain import Miner, Node, Block, Blockchain

# Creating the Blockchain

we initialize the blockchain ("ledger") with the genesis block. 

for our toy example, our ledger will be a python list containing Block objects.

In [5]:
genesis_block = Block(0, "none", 'laura pays $1 to steve', 'hello')
genesis_block.show()

>>> block index: 0
>>> previous hash: none
>>> current hash: hello
>>> transaction: laura pays $1 to steve


In [6]:
ledger = Blockchain(genesis_block)

## step 1: creating blocks
nodes create a new Block for every new transaction/record.

In [7]:
Node_Alice = Node(ledger)
Node_Bob = Node(ledger)

In [8]:
block1 = Node_Alice.generate_new_block('steve pays $2 to michelle')
block1.show()

>>> block index: 1
>>> previous hash: hello
>>> current hash: -- placeholder hash --
>>> transaction: steve pays $2 to michelle


## step 2: attempt to add new block to the ledger

a new Block cannot be added to the blockchain until its hash demonstrates **proof of work**. 

in our toy example, an acceptable hash is one where h(previous hash + transaction) starts with the character "c". h(x) in this case is sha1 hash function.

for example, sha1("hello World11") is 'cc8e9d1fbe83d1ce4b1bf546ebff3b735e4ff500'. since this hash output starts with the character "c", it will be considered to be a valid **proof of work** and thus its associated Block can be added to the ledger.

in practice, an acceptable hash, usually sha256, must start with three zeros (eg "000abfds..."). 

In [9]:
# can new block be added to blockchain?
Node_Alice.update_blockchain(block1)

>>> checking index... True
>>> checking hash sequence... True
>>> checking hash validity... False
>>> update failed


False

## step 3: miners demonstrate proof of work
through brute force, miners find the correct hash that satisfies conditions. once the correct hash is identified, it will be assigned to that block, thereby enabling addition to the ledger.

In [10]:
Miner_Adam = Miner()

In [11]:
good_hash = Miner_Adam.calc_hash_seal(block1)
print "acceptable hash:", good_hash

acceptable hash: 111111111111111111111111111111111111111111111111111111111111111111111


In [12]:
# miner updates the block with the hash
Miner_Adam.update_hash(block1, good_hash)

In [13]:
# with the new hash, verify new block can be added to blockchain
Node_Alice.update_blockchain(block1)

>>> checking index... True
>>> checking hash sequence... True
>>> checking hash validity... True
>>> successfully updated blockchain...


True

# simulate another transaction

In [14]:
block2 = Node_Alice.generate_new_block('michelle pays $1 to joanna')
block2.show()

>>> block index: 2
>>> previous hash: 111111111111111111111111111111111111111111111111111111111111111111111
>>> current hash: -- placeholder hash --
>>> transaction: michelle pays $1 to joanna


In [15]:
Node_Alice.check_block(block2)  # verify new block can be added to blockchain

>>> checking index... True
>>> checking hash sequence... True
>>> checking hash validity... False


False

In [16]:
good_hash = Miner_Adam.calc_hash_seal(block2)
print "good_hash:", good_hash

good_hash: 11111111111


In [17]:
# since we've found the seal, we can update the block, thus approving it for blockchain addition
Miner_Adam.update_hash(block2, good_hash)

In [18]:
Node_Alice.check_block(block2)  # verify new block can be added to blockchain

>>> checking index... True
>>> checking hash sequence... True
>>> checking hash validity... True


True

In [19]:
# with the new hash, verify new block can be added to blockchain
Node_Alice.update_blockchain(block2)

>>> checking index... True
>>> checking hash sequence... True
>>> checking hash validity... True
>>> successfully updated blockchain...


True

In [20]:
# examine history
for blocks in Node_Alice.blockchain_copy.blockchain:
    print blocks.fetch_transaction()

transaction 0 : laura pays $1 to steve
transaction 1 : steve pays $2 to michelle
transaction 2 : michelle pays $1 to joanna


In [21]:
# see if record 2 is valid
print "hash signatures match?", 
Node_Alice.blockchain_copy.blockchain[1].curr_hash == Miner.calc_hash_seal(Node_Alice.blockchain_copy.blockchain[1])

hash signatures match?

True

# what is record 1 was altered?

In [22]:
# examine history
for blocks in Node_Alice.blockchain_copy.blockchain:
    print blocks.fetch_transaction()

transaction 0 : laura pays $1 to steve
transaction 1 : steve pays $2 to michelle
transaction 2 : michelle pays $1 to joanna


In [23]:
Node_Alice.blockchain_copy.blockchain[1].data = "dummy data"

In [24]:
# examine history
for blocks in Node_Alice.blockchain_copy.blockchain:
    print blocks.fetch_transaction()

transaction 0 : laura pays $1 to steve
transaction 1 : dummy data
transaction 2 : michelle pays $1 to joanna


In [25]:
# see if record 2 is valid
print "hash signatures match?", 
Node_Alice.blockchain_copy.blockchain[1].curr_hash == Miner.calc_hash_seal(Node_Alice.blockchain_copy.blockchain[1])

hash signatures match?

False